In [1]:
import pandas as pd
from dateparser import parse
import requests as rq
import tqdm
import pyarrow
from password import *

In [2]:
fest = pd.read_parquet('../data/df_fest_clean.parquet')

In [3]:
fest.head()

,Festival,url,Cartel,Start_Date,End_Date,Location,info
0,Mallorca Live Festival 2023,https://www.songkick.com/es/festivals/1520684-...,"[Black Eyed Peas, Chemical Brothers, The Dandy...",jueves 18 mayo 2023,sábado 20 mayo 2023,Antic Aquapark,"Camí Cala Figuera, 1\n07181\nCalvia, Spain\nww..."
1,Primavera a la Ciutat 2023,https://www.songkick.com/es/festivals/3452756-...,"[Jake Bugg, Pet Shop Boys, Confidence Man, La ...",martes 30 mayo 2023,miércoles 31 mayo 2023,Sala Apolo,"C/ Nou De La Rambla 113\n08004\nBarcelona, Spa..."
2,Primavera In The City - Madrid 2023,https://www.songkick.com/es/festivals/3500869-...,"[Bleachers, Julia Jacklin, PUP, Black Country,...",lunes 05 junio 2023,domingo 11 junio 2023,Unknown,Unknown
3,Primavera Sound Festival 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",jueves 08 junio 2023,domingo 11 junio 2023,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
4,Primavera Pack 2023,https://www.songkick.com/es/festivals/3495900-...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",jueves 08 junio 2023,sábado 10 junio 2023,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."


In [4]:
fest.Start_Date = fest.Start_Date.apply(lambda x: parse(x,languages=['es']))
fest.End_Date = fest.End_Date.apply(lambda x: parse(x,languages=['es']))

In [5]:
fest[fest.Festival.str.contains('Primavera')]

,Festival,url,Cartel,Start_Date,End_Date,Location,info
1,Primavera a la Ciutat 2023,https://www.songkick.com/es/festivals/3452756-...,"[Jake Bugg, Pet Shop Boys, Confidence Man, La ...",2023-05-30,2023-05-31,Sala Apolo,"C/ Nou De La Rambla 113\n08004\nBarcelona, Spa..."
2,Primavera In The City - Madrid 2023,https://www.songkick.com/es/festivals/3500869-...,"[Bleachers, Julia Jacklin, PUP, Black Country,...",2023-06-05,2023-06-11,Unknown,Unknown
3,Primavera Sound Festival 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-11,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
4,Primavera Pack 2023,https://www.songkick.com/es/festivals/3495900-...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-08,2023-06-10,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
391,Primavera Sound Madrid 2023,https://www.songkick.com/es/festivals/3495787-...,"[Ganges, Dharmacide]",2023-06-05,2023-06-10,Ciudad del Rock,"Autovía A3: Sentido Valencia Salida 33A, 35\n2..."
404,Primavera Sound Barcelona 2023,https://www.songkick.com/es/festivals/618-prim...,"[Kendrick Lamar, Calvin Harris, Skrillex, Hals...",2023-06-01,2023-06-03,Parc del Fòrum,"Plaça de Llevant\n08019\nBarcelona, Spain\n93 ..."
476,Primavera Sound Porto 2023,https://www.songkick.com/es/festivals/1233638-...,"[St. Vincent, Sparks]",2023-06-07,2023-06-10,Parque da Cidade,Estrada Interior da Circunvalação 15443\n3700\...


In [6]:
' '.join(fest['info'][0].split()[:-1])

'Camí Cala Figuera, 1 07181 Calvia, Spain'

In [7]:
fest['info'] = fest['info'].apply(lambda x :' '.join(x.split()))

In [8]:
prueba = fest[fest['info'].str.contains('www')]['info'][0]

In [9]:
prueba = prueba.replace(' ',r'%20')

In [10]:
prueba

'Camí%20Cala%20Figuera,%201%2007181%20Calvia,%20Spain%20www.mallorcalivemusic.com'

In [11]:
url = f'https://maps.googleapis.com/maps/api/geocode/json?address={prueba}&key={API_KEY_GOOGLE}'
response = rq.get(url)
response

<Response [200]>

In [20]:
response.json()['results'][0]['address_components'][-2]['long_name']

'Spain'

In [21]:
def get_coordinates(x):
    global API_KEY_GOOGLE
    address = x.replace(' ','%20')
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY_GOOGLE}'
    try:
        response = rq.get(url)
        lat, long = response.json()['results'][0]['geometry']['location'].values()
        country = response.json()['results'][0]['address_components'][-2]['long_name']
        return {'Coordenadas':{'Latitud': lat,
                               'Longitud': long},
                'País':country}
    except:
        return {'Coordenadas':{'Latitud': None,
                               'Longitud': None},
                'País':None}

In [22]:
coord = fest['info'].apply(get_coordinates)

In [27]:
response.json()

{'results': [{'address_components': [{'long_name': '1',
     'short_name': '1',
     'types': ['street_number']},
    {'long_name': 'Camí Cala Figuera',
     'short_name': 'Camí Cala Figuera',
     'types': ['route']},
    {'long_name': 'Calvià',
     'short_name': 'Calvià',
     'types': ['locality', 'political']},
    {'long_name': 'Illes Balears',
     'short_name': 'PM',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Illes Balears',
     'short_name': 'IB',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Spain',
     'short_name': 'ES',
     'types': ['country', 'political']},
    {'long_name': '07181', 'short_name': '07181', 'types': ['postal_code']}],
   'formatted_address': 'Camí Cala Figuera, 1, 07181 Calvià, Illes Balears, Spain',
   'geometry': {'location': {'lat': 39.5068942, 'lng': 2.5193299},
    'location_type': 'ROOFTOP',
    'viewport': {'northeast': {'lat': 39.5082444302915,
      'lng': 2.520662980291501

In [53]:
coord = pd.DataFrame(coord)
coord.columns=['Coordenadas']
coord

,Coordenadas
0,"{'Latitud': 39.5068942, 'Longitud': 2.5193299}"
1,"{'Latitud': 41.3743015, 'Longitud': 2.1696845}"
2,"{'Latitud': None, 'Longitud': None}"
3,"{'Latitud': 39.4706902, 'Longitud': -0.4272878}"
4,"{'Latitud': 39.4706902, 'Longitud': -0.4272878}"
...,...
475,"{'Latitud': 41.8872258, 'Longitud': -8.7864370..."
476,"{'Latitud': 41.1713817, 'Longitud': -8.6765188}"
477,"{'Latitud': 38.6987186, 'Longitud': -9.2381618}"
478,"{'Latitud': 47.55308489999999, 'Longitud': 19...."


In [56]:
fest = pd.concat([fest,
                  coord,
                  pd.DataFrame(coord.Coordenadas.to_list())],axis=1)

In [62]:
fest[fest.Longitud.isna()].Festival.tolist()

['Primavera In The City - Madrid 2023',
 'Sonar 2023',
 'Vida Festival 2023',
 'Brunch In the Park - Barcelona 2023',
 'Dreambeach 2023',
 'Sonic Temple Art & Music Festival 2023',
 'Louder Than Life Festival 2023',
 'SXSW Sydney 2023',
 'Wave-Gotik-Treffen 2023',
 'Jazzopen Stuttgart 2023',
 'Lollapalooza Berlin 2023',
 'Santa Teresa 2023',
 'Montreal Jazz Fest 2023',
 'Ambleside Music Festival 2023',
 'Gop Tun Festival 2023',
 'Só Track Boa Festival - SP 2023',
 'Best of Blues and Rock Festival 2023',
 'Vintage is a Festival 2023',
 'Vintage is a Festival 2023',
 'PunkSpark 2023',
 'Dream Machine Bali 2023',
 'Prambanan Jazz Festival 2023',
 'Soundrenaline 2023',
 'Pestapora 2023',
 'Joyland Festival 2023',
 'Komm schon Alter Festival 2023',
 'Lowlands Festival 2023',
 'Les Nuits de Fourvière 2023',
 'We Love Green 2023',
 'Festival de Nîmes 2023',
 'İnstagram: ilhxn444 2023',
 'No Label Festival 2023',
 'ig : @fatinigga 2023',
 'CHILLOUT FESTIVAL 2023',
 'Kapadokya Müzik Festivali 2

In [58]:
fest.isna()

,Festival,url,Cartel,Start_Date,End_Date,Location,info,Coordenadas,Latitud,Longitud
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
475,False,False,False,False,False,False,False,False,False,False
476,False,False,False,False,False,False,False,False,False,False
477,False,False,False,False,False,False,False,False,False,False
478,False,False,False,False,False,False,False,False,False,False


In [64]:
fest.to_parquet('../data/fest_coord.parquet',index=False)

In [65]:
import pymongo

In [66]:
cursor = pymongo.MongoClient('mongodb://localhost:27017')

In [68]:
db_fest = cursor.Festivales

In [70]:
fest_coord = db_fest.fest_coord

In [72]:
fest.Cartel = fest.Cartel.apply(list)
fest_coord.insert_many(fest.to_dict(orient='records'))